<a href="https://colab.research.google.com/github/NPCA-TEAM/COVID-19/blob/main/Scripts/PREDICAO_MODELOS_covid_obitos_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação, importação de bibliotecas e inicialização

##Instalação das bibliotecas

In [ ]:
#Inicia permitindo acesso ao GDrive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyyaml==5.4.1

In [ ]:
!pip install darts

In [ ]:
!pip install matplotlib==3.1.3  #é necessário restart runtime 

In [ ]:
!pip install pytorch-lightning

##Importação das Bibliotecas

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import glob
import os
import sys
import time
import torch
import json

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

from datetime import datetime

from darts import timeseries
from darts.dataprocessing.transformers import Scaler
from darts import concatenate

from darts.models import (    
    NBEATSModel,
    TCNModel,
    TransformerModel,
    TFTModel,
    NHiTSModel
    )

from darts.metrics import mape, rmse, r2_score
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.missing_values import fill_missing_values
from darts.models import KalmanFilter

from darts.utils.likelihood_models import (
    GaussianLikelihood, 
    QuantileRegression 
    )

import warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.CRITICAL)
torch.manual_seed(1); np.random.seed(1)  # for reproducibility

#Dados e campos de entrada do framework


In [ ]:
#Campos de entrada do framework
NAME_SERIE_OCCURRENCE_DEATH = "Obitos_DataOcorrencia_MM_atual_PA"   #Variavel onde será adicionada o nome para a serie a ser analisada
NAME_SERIE_PUBLICATION_DEATH = "Obitos_Publicacao_MM_atual_PA"   #Variavel onde será adicionada o nome para a serie a ser analisada

#Número de anos para validação/teste. Similar a quantidade de dados de teste
#Se num_days_of_slice_val = 1, indica que o ultimo ano da serie será usado para testar o modelo
#Também indica a fatia usada como validação na fase de treinamento
VALIDATION_DAYS_SLICED = 7

#Número que indica o tamanho da fatia usada como treino na fase de treinamento. 
#Logo, o processo disposto neste script é baseado no metodo de janela deslizante.
WORKOUT_DAYS_SLICED = 30

#Numero de passos da previsão a frente da serie
FORECAST_DAYS = 7
 
# Arquivo com a base de dados
PATH_DATASET = '/content/drive/MyDrive/NPCA - COVID/_OBITOS/DataSet/' 

#Pasta para salvamento do modelo
PATH_MODELS = '/content/drive/MyDrive/NPCA - COVID/_OBITOS/Models/'
PATH_FIGS = '/content/drive/MyDrive/NPCA - COVID/_OBITOS/Figs/'
PATH_CSV = '/content/drive/MyDrive/NPCA - COVID/_OBITOS/ForecastsOutputModel/'

#Leitura do dataset

In [ ]:
#Seleciona o arquivo da base
FILE_LIST = glob.glob(PATH_DATASET + '*')

#Seleciona o ultimo arquivo que foi criado e adiciona na pasta
FILE_PATH = max(FILE_LIST, key=os.path.getctime)

#Faz leitura do arquivo excel como datafrade
DATAFRAME = pd.read_excel(FILE_PATH, sheet_name=0)

DATAFRAME

#Seleção das variáveis

In [ ]:
#Seleção das variáveis alvo
OCCURRENCE_DEATH = DATAFRAME.loc[:, ['data', 'Obitos_DataOcorrencia_MM_atual_PA']]
PUBLICATION_DEATH = DATAFRAME.loc[:, ['data', 'Obitos_Publicacao_MM_atual_PA']]

#Seleção das variáveis auxiliares
#OCURRENCE_7DAYS__DEATH = DATAFRAME.loc[:, ['data', 'Obitos_DataOcorrencia_MM_7dias_PA']].fillna(0) 
#PUBLICATION_7DAYS = DATAFRAME.loc[:, ['data', 'Obitos_DataPublicacao_MM_7dias_PA']].fillna(0)
OCURRENCE_14DAYS_DEATH = DATAFRAME.loc[:, ['data', 'Obito_DataOcorrencia_MM_14dias_PA']].fillna(0)
PUBLICATION_14DAYS_DEATH = DATAFRAME.loc[:, ['data', 'Obitos_DataPublicacao_MM_14dias_PA']].fillna(0)
#OBS: No predição historica usaremos somete o 14Dias. 

VACCINATION_1DOSE = DATAFRAME.loc[:, ['data', 'Vacinacao_Dose1_%decimal_PA']].fillna(0)
VACCINATION_2DOSE = DATAFRAME.loc[:, ['data', 'Vacinacao_Dose2_%decimal_PA']].fillna(0)
VACCINATION_3DOSE = DATAFRAME.loc[:, ['data', 'Vacinacao_Dose3_%decimal_PA']].fillna(0)

#Renomeia variaveis.
OCCURRENCE_7PREVIOUSDAYS_DEATH = OCURRENCE_14DAYS_DEATH.rename(columns = {'Obito_DataOcorrencia_MM_14dias_PA':'Obito_DataOcorrencia_MM_7diasAnteriores_PA'}, inplace = False)
PUBLICACAO_7PREVIOUSDAYS_DEATH = PUBLICATION_14DAYS_DEATH.rename(columns = {'Obitos_DataPublicacao_MM_14dias_PA':'Obitos_DataPublicacao_MM_7diasAnteriores_PA'}, inplace = False)

In [ ]:
#Seleção das variáveis auxiliares de casos
SYMPTOM_CASES = DATAFRAME.loc[:, ['data', 'Casos_DataSintoma_MM_atual_PA']]
PUBLICATION_CASES = DATAFRAME.loc[:, ['data', 'Casos_Publicacao_MM_atual_PA']]

SYMPTOM_14DAYS_CASES = DATAFRAME.loc[:, ['data', 'Casos_DataSintoma_MM_14dias_PA']].fillna(0)
PUBLICATION_14DAYS_CASES = DATAFRAME.loc[:, ['data', 'Casos_Publicacao_MM_14dias_PA']].fillna(0)
#OBS: No predição historica usaremos somente os 14 Dias. 

#Renomeia variaveis.
SYMPTOM_7PREVIOUSDAYS_CASES = SYMPTOM_14DAYS_CASES.rename(columns = {'Casos_DataSintoma_MM_14dias_PA':'Casos_DataSintoma_MM_7diasAnteriores_PA'}, inplace = False)
PUBLICATION_7PREVIOUSDAYS_CASES = PUBLICATION_14DAYS_CASES.rename(columns = {'Casos_Publicacao_MM_14dias_PA':'Casos_Publicacao_MM_7diasAnteriores_PA'}, inplace = False)

#Instanciação das séries temporais

In [ ]:
# criação das séries temporais alvos
serie_occurrence_death = timeseries.TimeSeries.from_dataframe(df=OCCURRENCE_DEATH, time_col='data')
serie_publication_death = timeseries.TimeSeries.from_dataframe(df=PUBLICATION_DEATH, time_col='data')

plt.figure(figsize = (10, 6))

serie_occurrence_death.plot()
serie_publication_death.plot()

In [ ]:
# criação das séries temporais auxiliares de obitos
serie_occurrence_death_7previousdays = timeseries.TimeSeries.from_dataframe(df=OCCURRENCE_7PREVIOUSDAYS_DEATH, time_col='data')
serie_publication_death_7previousdays = timeseries.TimeSeries.from_dataframe(df=PUBLICACAO_7PREVIOUSDAYS_DEATH, time_col='data')

plt.figure(figsize = (10, 6))

serie_occurrence_death_7previousdays.plot()
serie_publication_death_7previousdays.plot()

In [ ]:
series_symptom_cases = timeseries.TimeSeries.from_dataframe(df=SYMPTOM_CASES, time_col='data')
series_publication_cases = timeseries.TimeSeries.from_dataframe(df=PUBLICATION_CASES, time_col='data')

plt.figure(figsize = (14, 10))

series_symptom_cases.plot()
series_publication_cases.plot()

In [ ]:
series_symptom_cases_7previousdays = timeseries.TimeSeries.from_dataframe(df=SYMPTOM_7PREVIOUSDAYS_CASES, time_col='data')
series_publication_cases_7previousdays = timeseries.TimeSeries.from_dataframe(df=PUBLICATION_7PREVIOUSDAYS_CASES, time_col='data')

plt.figure(figsize = (14, 10))

series_symptom_cases_7previousdays.plot()
series_publication_cases_7previousdays.plot()

In [ ]:
# criação das séries temporais auxiliares de vacinação
serie_vaccination_1Dose = timeseries.TimeSeries.from_dataframe(df=VACCINATION_1DOSE, time_col='data')
serie_vaccination_2Dose = timeseries.TimeSeries.from_dataframe(df=VACCINATION_2DOSE, time_col='data')
serie_vaccination_3Dose = timeseries.TimeSeries.from_dataframe(df=VACCINATION_3DOSE, time_col='data')

plt.figure(figsize = (10, 6))

serie_vaccination_1Dose.plot()
serie_vaccination_2Dose.plot()
serie_vaccination_3Dose.plot()

#Filtro de suaviação das variáveis

In [ ]:
def filter_for_suavization(serie):
  filterKalman = KalmanFilter(dim_x = 1)
  filterKalman.fit(serie)
  serieTemp_filtered = filterKalman.filter(serie)

  return serieTemp_filtered

In [ ]:
#Suavização das series alvos (tratar outliers)
smoothed_occurrence_death_serie = filter_for_suavization(serie_occurrence_death)
smoothed_publication_death_serie = filter_for_suavization(serie_publication_death)

plt.figure(figsize = (10, 6))

serie_occurrence_death.plot(label="Ocorrencia Obitos")
smoothed_occurrence_death_serie.plot(label="Ocorrencia Obitos Suav")
serie_publication_death.plot(label="Publicação Obitos")
smoothed_publication_death_serie.plot(label="Publicação Obitos Suav")

#Escalar (normalização entre 0 e 1)

In [ ]:
def process_scaler(serie):
  Scaler_obj = Scaler()
  serieTemp_scaled = Scaler_obj.fit_transform(serie)

  return serieTemp_scaled, Scaler_obj

In [ ]:
#pré-processamento e normalização [0, 1]
normalized_occurrence_serie_death, SCALER_occurrence_death = process_scaler(smoothed_occurrence_death_serie)
normalized_publication_serie_death, SCALER_publication_death = process_scaler(smoothed_publication_death_serie)

occurrence_series_death_7days_ago_normalized, SCALER_occurrence_death_14days = process_scaler(serie_occurrence_death_7previousdays)
publication_series_death_7days_ago_normalized, SCALER_publication_death_14days = process_scaler(serie_publication_death_7previousdays)

In [ ]:
plt.figure(figsize = (10, 6))

normalized_occurrence_serie_death.plot()
normalized_publication_serie_death.plot()

occurrence_series_death_7days_ago_normalized.plot()
publication_series_death_7days_ago_normalized.plot()

In [ ]:
#pré-processamento e normalização [0, 1]
normalized_symptom_cases_series, Scaler_symptom = process_scaler(series_symptom_cases)
normalized_publication_cases_series, Scaler_publication = process_scaler(series_publication_cases)

symptom_cases_series_7days_ago_normalized, Scaler_symptom_14days = process_scaler(series_symptom_cases_7previousdays)
publication_cases_series_7days_ago_normalized, Scaler_publication_14days = process_scaler(series_publication_cases_7previousdays)

In [ ]:
plt.figure(figsize = (14, 10))

normalized_symptom_cases_series.plot()
normalized_publication_cases_series.plot()

symptom_cases_series_7days_ago_normalized.plot()
publication_cases_series_7days_ago_normalized.plot()

#Definição de Covariáveis

In [ ]:
def define_covariate_dates(serie_temp_normalizada):  #modificar nomes
  #Definir covariavel ano
  scaler_CovarYear, scaler_CovarMonth = Scaler(), Scaler()
  covariate_y = datetime_attribute_timeseries(serie_temp_normalizada, attribute='year', add_length = FORECAST_DAYS)
  covariate_y = scaler_CovarYear.fit_transform(covariate_y)

  #Definir covariavel mes
  covariate_m = datetime_attribute_timeseries(serie_temp_normalizada, attribute='month', add_length = FORECAST_DAYS)
  covariate_m = scaler_CovarMonth.fit_transform(covariate_m)

  #Definir covariavel day of week
  covariate_dw = datetime_attribute_timeseries(serie_temp_normalizada, attribute='dayofweek', add_length = FORECAST_DAYS)
  covariate_dw = scaler_CovarMonth.fit_transform(covariate_dw)

  return covariate_y, covariate_m, covariate_dw

In [ ]:
#definição das covariáveis
covariate_year_occurrence_death, covariate_month_occurrence_death, covariate_dayweek_occurrence_death = define_covariate_dates(normalized_occurrence_serie_death)

In [ ]:
#Visualização das series covariaveis de data
plt.figure(figsize = (14, 10))
covariate_year_occurrence_death.plot(label='Covariates by Year')
covariate_month_occurrence_death.plot(label='Covariates by Month')

plt.title(label='Series of Covariates by Month and Year (Death)', fontsize=16)
plt.savefig(PATH_FIGS + 'Series of Covariates by Month and Year (Death).png', dpi=300)

In [ ]:
plt.figure(figsize = (14, 10))
covariate_dayweek_occurrence_death[-30:].plot(label='Day of Week' ,marker='o')

plt.title(label='Series of Covariates by Days of The Week (Death)', fontsize=16)
plt.savefig(PATH_FIGS + 'Series of Covariates by Days of The Week (Death).png', dpi=300)

#Divisão do dataset de treino e validação

In [ ]:
#Dividir em dados de treino e teste
def split_train_val_series(serie):
  s_train, s_val = serie[:-VALIDATION_DAYS_SLICED], serie[-VALIDATION_DAYS_SLICED:]
  
  return s_train, s_val

In [ ]:
#divisão dos dados de treino e testes
serieTrain_occurrence_death, serieVal_occurrence_death = split_train_val_series(normalized_occurrence_serie_death)
serieTrain_publication_death, serieVal_publication_death = split_train_val_series(normalized_publication_serie_death)

In [ ]:
#Visualização das Series Alvos
plt.figure(figsize = (10, 6))

serieTrain_occurrence_death[-30:].plot()   # plotagem dos ultimos 30 dias para treino
serieVal_occurrence_death.plot()
serieTrain_publication_death[-30:].plot()   # plotagem dos ultimos 30 dias para treino
serieVal_publication_death.plot()


#Predição dos modelos

##Preparação para predição

In [ ]:
#Junção das variaveis alvo e coovariaveis
series_Train = serieTrain_occurrence_death.stack(serieTrain_publication_death)

covariatesDate = covariate_year_occurrence_death.stack(covariate_month_occurrence_death.stack(covariate_dayweek_occurrence_death))
covariatesCases = normalized_symptom_cases_series.stack(normalized_publication_cases_series.stack(symptom_cases_series_7days_ago_normalized.stack(publication_cases_series_7days_ago_normalized)))
covariatesDeaths = occurrence_series_death_7days_ago_normalized.stack(publication_series_death_7days_ago_normalized)
covariatesVacination = serie_vaccination_1Dose.stack(serie_vaccination_2Dose.stack(serie_vaccination_3Dose))

#O [:-FORECAST_DAYS] é um corte na covariatesDate para que as covariaveis tenham o mesmo numero de dias.
covariates = covariatesDate[:-FORECAST_DAYS].stack(covariatesDeaths.stack(covariatesCases.stack(covariatesVacination)))

plt.figure(figsize = (10, 6))
series_Train.plot()
covariates.plot()

##Importação dos modelos para predição

In [ ]:
#Seleciona os arquivos 
file_list_models = glob.glob(PATH_MODELS + '*.tar')

#Seleciona os arquivos adicionados mais recentementes na pasta
file_path_models = sorted(file_list_models, key=os.path.getctime, reverse=True)

In [ ]:
#Filtrar caominhos dos modelos que tem extensão pth.tar validas
current_year = str(datetime.today().year)
file_path_models_currents = list(filter(lambda x: current_year+'.pth.tar' in x, file_path_models))

In [ ]:
#Selecionar o caminho dos modelos
NBEATS_model_path = list(filter(lambda x: 'NBEATS' in x, file_path_models_currents))[0]
model_NBEATSModel = NBEATSModel.load_model(NBEATS_model_path)
print(NBEATS_model_path)

TCN_model_path = list(filter(lambda x: 'TCN' in x, file_path_models_currents))[0]
model_TCNModel = TCNModel.load_model(TCN_model_path)
print(TCN_model_path)

TFT_model_path = list(filter(lambda x: 'TFT' in x, file_path_models_currents))[0]
model_TFTModel = TFTModel.load_model(TFT_model_path)
print(TFT_model_path)

TRANSFORMER_model_path = list(filter(lambda x: 'TRANSFORMER' in x, file_path_models_currents))[0]
model_TRANSFORMERModel = TransformerModel.load_model(TRANSFORMER_model_path)
print(TRANSFORMER_model_path)

NHITS_model_path = list(filter(lambda x: 'NHITS' in x, file_path_models_currents))[0]
model_NHITSModel = NHiTSModel.load_model(NHITS_model_path)
print(NHITS_model_path)

##Acelerador COLAB

In [ ]:
# Utilizar o acelerador de GPU do colab (para treinamento)
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))
  tf.device('/device:GPU:0')
  print()

  physical_devices = tf.config.experimental.list_logical_devices('GPU')
  print("GPUs available: ", physical_devices)
  print("GPUs available: ", len(physical_devices))
  print(physical_devices[0].name)
  print()

  # Qual placa de video GPU estou usando?
  from tensorflow.python.client import device_lib
  device_lib.list_local_devices()
  print(device_lib.list_local_devices())

##Predição sobre dados de treino

In [ ]:
from timeit import default_timer as timer
start = timer()

In [ ]:
def historical_forecasts(model, series_target, covariates):
  serie_hist_fcast = model.historical_forecasts(
      series = series_target,
      past_covariates = covariates,
      start = WORKOUT_DAYS_SLICED,
      forecast_horizon = VALIDATION_DAYS_SLICED,
      stride = 1,
      num_samples = 50,  #pode ser reduzido para teste. valor padrão: 50
      retrain = False,
      verbose = False 
      )
  
  return serie_hist_fcast

In [ ]:
historical_forecasts_NBEATS = historical_forecasts(model_NBEATSModel, series_Train, covariates)

In [ ]:
historical_forecasts_TCN = historical_forecasts(model_TCNModel, series_Train, covariates)

In [ ]:
historical_forecasts_TFT = historical_forecasts(model_TFTModel, series_Train, covariates)

In [ ]:
historical_forecasts_Transformer = historical_forecasts(model_TRANSFORMERModel, series_Train, covariates)

In [ ]:
historical_forecasts_NHITS = historical_forecasts(model_NHITSModel, series_Train, covariates)

##Predição sobre dados de validação

In [ ]:
def serie_predict(model, series_target, covariates):
  serie_predicted = model.predict(
      n = FORECAST_DAYS,
      series = series_target,
      past_covariates = covariates,
      num_samples = 50,
      n_jobs = -1) 
  
  return serie_predicted

In [ ]:
serie_predict_NBEATS = serie_predict(model_NBEATSModel, series_Train, covariates)

In [ ]:
serie_predict_TCN = serie_predict(model_TCNModel, series_Train, covariates)

In [ ]:
serie_predict_TFT = serie_predict(model_TFTModel, series_Train, covariates)

In [ ]:
serie_predict_Transformer = serie_predict(model_TRANSFORMERModel, series_Train, covariates)

In [ ]:
serie_predict_NHITS = serie_predict(model_NHITSModel, series_Train, covariates)

In [ ]:
end = timer()
print("Tempo decorrido de predição: {:.2f}".format(end - start))

##Visualização das predições


In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (10, 7))
series_Train.plot()
serieVal_occurrence_death.plot(label ='serieVal_ocorrencia - Real')
serieVal_publication_death.plot(label ='serieVal_publication - Real')

historical_forecasts_NBEATS.plot()
serie_predict_NBEATS.plot()

In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (10, 7))
series_Train[-30:].plot()
serieVal_occurrence_death.plot(label ='serieVal_ocorrencia - Real')
serieVal_publication_death.plot(label ='serieVal_publication - Real')

historical_forecasts_NBEATS[-30:].plot()
serie_predict_NBEATS.plot()

In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (10, 7))
series_Train[-30:].plot()
serieVal_occurrence_death.plot(label ='serieVal_ocorrencia - Real')
serieVal_publication_death.plot(label ='serieVal_publication - Real')

historical_forecasts_TCN[-30:].plot()
serie_predict_TCN.plot()

In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (10, 7))
series_Train.plot()
serieVal_occurrence_death.plot(label ='serieVal_ocorrencia - Real')
serieVal_publication_death.plot(label ='serieVal_publication - Real')

historical_forecasts_TFT.plot()
serie_predict_TFT.plot()

In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (10, 7))
series_Train.plot()
serieVal_occurrence_death.plot(label ='serieVal_ocorrencia - Real')
serieVal_publication_death.plot(label ='serieVal_publication - Real')

historical_forecasts_Transformer.plot()
serie_predict_Transformer.plot()

In [ ]:
  #Visualizar series alvos e predições
plt.figure(figsize = (10, 7))
series_Train[-30:].plot()
serieVal_occurrence_death.plot(label ='serieVal_ocorrencia - Real')
serieVal_publication_death.plot(label ='serieVal_publication - Real')

historical_forecasts_NHITS[-30:].plot()
serie_predict_NHITS.plot()

#Geração e Visualização da mediana das series

In [ ]:
def series_view(historical_forecasts, serie_predict):
  occurrence_histfcast_median = historical_forecasts['Obitos_DataOcorrencia_MM_atual_PA'].median()
  publication_histfcast_median = historical_forecasts['Obitos_Publicacao_MM_atual_PA'].median()

  occurrence_pred_median = serie_predict['Obitos_DataOcorrencia_MM_atual_PA'].median()
  publication_pred_median = serie_predict['Obitos_Publicacao_MM_atual_PA'].median()

  return occurrence_histfcast_median, publication_histfcast_median, occurrence_pred_median, publication_pred_median

In [ ]:
NBEATS_occurrence_histfcast_median, NBEATS_publication_histfcast_median, NBEATS_occurrence_pred_median, NBEATS_publication_pred_median = series_view(historical_forecasts_NBEATS, serie_predict_NBEATS)

#Aumento do tamanho da figura (x, y)
plt.figure(figsize = (14, 9))

NBEATS_occurrence_histfcast_median.plot(label='NBEATS_Mediana_Previsão_Histórica Obitos_DataOcorrencia_MM_atual_PA')
NBEATS_publication_histfcast_median.plot(label='NBEATS_Mediana_Previsão_Histórica Obitos_Publicacao_MM_atual_PA')
NBEATS_occurrence_pred_median.plot()
NBEATS_publication_pred_median.plot()

In [ ]:
TCN_occurrence_histfcast_median, TCN_publication_histfcast_median, TCN_occurrence_pred_median, TCN_publication_pred_median = series_view(historical_forecasts_TCN, serie_predict_TCN)

#Aumento do tamanho da figura (x, y)
plt.figure(figsize = (15, 10))

TCN_occurrence_histfcast_median.plot(label='TCN_Mediana_Previsão_Histórica Obitos_DataOcorrencia_MM_atual_PA')
TCN_publication_histfcast_median.plot(label='TCN_Mediana_Previsão_Histórica Obitos_Publicacao_MM_atual_PA')
TCN_occurrence_pred_median.plot()
TCN_publication_pred_median.plot()

In [ ]:
TFT_occurrence_histfcast_median, TFT_publication_histfcast_median, TFT_occurrence_pred_median, TFT_publication_pred_median = series_view(historical_forecasts_TFT, serie_predict_TFT)

#Aumento do tamanho da figura (x, y)
plt.figure(figsize = (15, 10))

TFT_occurrence_histfcast_median.plot(label='TFT_Mediana_Previsão_Histórica Obitos_Dataoccurrence_MM_atual_PA')
TFT_publication_histfcast_median.plot(label='TFT_Mediana_Previsão_Histórica Obitos_Publicacao_MM_atual_PA')
TFT_occurrence_pred_median.plot()
TFT_publication_pred_median.plot()

In [ ]:
Transformer_occurrence_histfcast_median, Transformer_publication_histfcast_median, Transformer_occurrence_pred_median, Transformer_publication_pred_median = series_view(historical_forecasts_Transformer, serie_predict_Transformer)

#Aumento do tamanho da figura (x, y)
plt.figure(figsize = (15, 10))

Transformer_occurrence_histfcast_median.plot(label='TRANSFORMER_Mediana_Previsão_Histórica Obitos_DataOcorrencia_MM_atual_PA')
Transformer_publication_histfcast_median.plot(label='TRANSFORMER_Mediana_Previsão_Histórica Obitos_Publicacao_MM_atual_PA')
Transformer_occurrence_pred_median.plot()
Transformer_publication_pred_median.plot()

In [ ]:
NHITS_occurrence_histfcast_median, NHITS_publication_histfcast_median, NHITS_occurrence_pred_median, NHITS_publication_pred_median = series_view(historical_forecasts_NHITS, serie_predict_NHITS)

#Aumento do tamanho da figura (x, y)
plt.figure(figsize = (15, 10))

NHITS_occurrence_histfcast_median.plot(label='NHITS_Mediana_Previsão_Histórica Obitos_DataOcorrencia_MM_atual_PA')
NHITS_publication_histfcast_median.plot(label='NHITS_Mediana_Previsão_Histórica Obitos_Publicacao_MM_atual_PA')
NHITS_occurrence_pred_median.plot()
NHITS_publication_pred_median.plot()

#Re-escala dos dados gerados




In [ ]:
def process_re_scaling(Scaler_serie, serie_scaled): 
  #Volta para a escala original
  serie_back = Scaler_serie.inverse_transform(serie_scaled)  

  return serie_back

In [ ]:
#Re-escala e Visualização do alvo de publicação - NBEATS

NBEATS_serie_publication_histfcast_rescaled = process_re_scaling(SCALER_publication_death, NBEATS_publication_histfcast_median)
NBEATS_serie_publication_pred_rescaled = process_re_scaling(SCALER_publication_death, NBEATS_publication_pred_median)
NBEATS_serie_occurrence_histfcast_rescaled = process_re_scaling(SCALER_occurrence_death, NBEATS_occurrence_histfcast_median)
NBEATS_serie_occurrence_pred_rescaled = process_re_scaling(SCALER_occurrence_death, NBEATS_occurrence_pred_median)


#Re-escala e Visualização do alvo de publicação - TCN
TCN_serie_publication_histfcast_rescaled = process_re_scaling(SCALER_publication_death, TCN_publication_histfcast_median)
TCN_serie_publication_pred_rescaled = process_re_scaling(SCALER_publication_death, TCN_publication_pred_median)
TCN_serie_occurrence_histfcast_rescaled = process_re_scaling(SCALER_occurrence_death, TCN_occurrence_histfcast_median)
TCN_serie_occurrence_pred_rescaled = process_re_scaling(SCALER_occurrence_death, TCN_occurrence_pred_median) 


#Re-escala e Visualização do alvo de publicação - TFT
TFT_serie_publication_histfcast_rescaled = process_re_scaling(SCALER_publication_death, TFT_publication_histfcast_median)
TFT_serie_publication_pred_rescaled = process_re_scaling(SCALER_publication_death, TFT_publication_pred_median)
TFT_serie_occurrence_histfcast_rescaled = process_re_scaling(SCALER_occurrence_death, TFT_occurrence_histfcast_median)
TFT_serie_occurrence_pred_rescaled = process_re_scaling(SCALER_occurrence_death, TFT_occurrence_pred_median)


#Re-escala e Visualização do alvo de publicação - TRANSFORMER
Transformer_serie_publication_histfcast_rescaled = process_re_scaling(SCALER_publication_death, Transformer_publication_histfcast_median)
Transformer_serie_publication_pred_rescaled = process_re_scaling(SCALER_publication_death, Transformer_publication_pred_median)
Transformer_serie_occurrence_histfcast_rescaled = process_re_scaling(SCALER_occurrence_death, Transformer_occurrence_histfcast_median)
Transformer_serie_occurrence_pred_rescaled = process_re_scaling(SCALER_occurrence_death, Transformer_occurrence_pred_median)


#Re-escala e Visualização do alvo de publicação - NHITS
NHITS_serie_publication_histfcast_rescaled = process_re_scaling(SCALER_publication_death, NHITS_publication_histfcast_median)
NHITS_serie_publication_pred_rescaled = process_re_scaling(SCALER_publication_death, NHITS_publication_pred_median)
NHITS_serie_occurrence_histfcast_rescaled = process_re_scaling(SCALER_occurrence_death, NHITS_occurrence_histfcast_median)
NHITS_serie_occurrence_pred_rescaled = process_re_scaling(SCALER_occurrence_death, NHITS_occurrence_pred_median)

In [ ]:
title_7days = 'Moving average (7 days) forecast of deaths'
title_37days = 'Moving average (7 days) forecast of deaths (showing last 37 days)'

In [ ]:
plt.figure(figsize = (14, 10))

smoothed_publication_death_serie.plot(label='Current Publication')
NBEATS_serie_publication_histfcast_rescaled.plot(label='Publication Training Dataset Prediction') # Previsão publicação sobre Training Dataset
NBEATS_serie_publication_pred_rescaled.plot(label='Publication Validation Dataset Prediction') # Previsão publicação sobre Validation Dataset

smoothed_occurrence_death_serie.plot(label='Current Deaths')
NBEATS_serie_occurrence_histfcast_rescaled.plot(label='First Hits Prediction in Training Dataset') # Previsão primeiros sintomas sobre Training Dataset
NBEATS_serie_occurrence_pred_rescaled.plot(label='First hits prediction in validation dataset')

plt.title(label= title_7days + ' - NBEATS Model', fontsize=16) # Previsão de Casos (média móvel 7 dias) - Modelo NBEATS

plt.savefig(PATH_FIGS + title_7days +' - NBEATS_Model.png', dpi = 300)

In [ ]:
#Visualiação com os ultimos 37 dias. - NBEATS
plt.figure(figsize = (14, 10))

smoothed_publication_death_serie[-37:].plot(label='Current Publication')
NBEATS_serie_publication_histfcast_rescaled[-30:].plot(label='Publication Training Dataset Prediction')
NBEATS_serie_publication_pred_rescaled[-30:].plot(label='Publication Validation Dataset Prediction')

smoothed_occurrence_death_serie[-37:].plot(label='Current Deaths')
NBEATS_serie_occurrence_histfcast_rescaled[-30:].plot(label='First Hits Prediction in Training Dataset')
NBEATS_serie_occurrence_pred_rescaled[-30:].plot(label='First hits prediction in validation dataset')

plt.title(label= title_37days + ' - NBEATS Model', fontsize=16) 

plt.savefig(PATH_FIGS + title_37days + ' - NBEATS Model.png', dpi = 300)

In [ ]:
#Visualização do alvo de publicação
plt.figure(figsize = (14, 10))

smoothed_publication_death_serie.plot(label='Current Publication')
TCN_serie_publication_histfcast_rescaled.plot(label='Publication Training Dataset Prediction')
TCN_serie_publication_pred_rescaled.plot(label='Publication Validation Dataset Prediction') 

smoothed_occurrence_death_serie.plot(label='Current Deaths')
TCN_serie_occurrence_histfcast_rescaled.plot(label='First Hits Prediction in Training Dataset')
TCN_serie_occurrence_pred_rescaled.plot(label='First hits prediction in validation dataset')

plt.title(label= title_7days + ' - TCN Model', fontsize=16) # Previsão de Casos (média móvel 7 dias) - Modelo NBEATS

plt.savefig(PATH_FIGS + title_7days +' - TCN_Model.png', dpi = 300)

In [ ]:
#Visualiação com com os ultimos 37 dias. - TCN
plt.figure(figsize = (14, 10))

smoothed_publication_death_serie[-37:].plot(label='Current Publication')
TCN_serie_publication_histfcast_rescaled[-30:].plot(label='Publication Training Dataset Prediction')
TCN_serie_publication_pred_rescaled[-30:].plot(label='Publication Validation Dataset Prediction') 

smoothed_occurrence_death_serie[-37:].plot(label='Current Deaths')
TCN_serie_occurrence_histfcast_rescaled[-30:].plot(label='First Hits Prediction in Training Dataset')
TCN_serie_occurrence_pred_rescaled[-30:].plot(label='First Hits Prediction in Validation Dataset')

plt.title(label= title_37days + ' - TCN Model', fontsize=16)

plt.savefig(PATH_FIGS + title_37days +' - TCN_Model.png', dpi = 300)

In [ ]:
#Visualização do alvo de publicação
plt.figure(figsize = (14, 10))

smoothed_publication_death_serie.plot(label='Current Publication')
TFT_serie_publication_histfcast_rescaled.plot(label='Publication Training Dataset Prediction')
TFT_serie_publication_pred_rescaled.plot(label='Publication Validation Dataset Prediction') 

smoothed_occurrence_death_serie.plot(label='Current Deaths')
TFT_serie_occurrence_histfcast_rescaled.plot(label='First Hits Prediction in Training Dataset')
TFT_serie_occurrence_pred_rescaled.plot(label='First Hits Prediction in Validation Dataset')

plt.title(label= title_7days + ' - TFT Model', fontsize=16)

plt.savefig(PATH_FIGS + title_7days +' - TFT_Model.png', dpi = 300)

In [ ]:
#Visualiação com com os ultimos 37 dias. - TFT
plt.figure(figsize = (14, 10))

smoothed_publication_death_serie[-37:].plot(label='Current Publication')
TFT_serie_publication_histfcast_rescaled[-30:].plot(label='Publication Training Dataset Prediction')
TFT_serie_publication_pred_rescaled[-30:].plot(label='Publication Validation Dataset Prediction') 

smoothed_occurrence_death_serie[-37:].plot(label='Current Deaths')
TFT_serie_occurrence_histfcast_rescaled[-30:].plot(label='First Hits Prediction in Training Dataset')
TFT_serie_occurrence_pred_rescaled[-30:].plot(label='First Hits Prediction in Validation Dataset')

plt.title(label= title_37days + ' - TFT Model', fontsize=16)
plt.savefig(PATH_FIGS + title_37days +' - TFT_Model.png', dpi = 300)

In [ ]:
#Visualização do alvo de publicação
plt.figure(figsize = (14, 10))

smoothed_publication_death_serie.plot(label='Current Publication')
Transformer_serie_publication_histfcast_rescaled.plot(label='Publication Training Dataset Prediction')
Transformer_serie_publication_pred_rescaled.plot(label='Publication Validation Dataset Prediction') 

smoothed_occurrence_death_serie.plot(label='Current Deaths')
Transformer_serie_occurrence_histfcast_rescaled.plot(label='First Hits Prediction in Training Dataset')
Transformer_serie_occurrence_pred_rescaled.plot(label='First Hits Prediction in Validation Dataset')

plt.title(label= title_7days + ' - TRANSFORMER Model', fontsize=16)

plt.savefig(PATH_FIGS + title_7days +' - TRANSFORMER_Model.png', dpi = 300)

In [ ]:
#Visualiação com com os ultimos 37 dias. - Transformer
plt.figure(figsize = (14, 10))

smoothed_publication_death_serie[-37:].plot(label='Current Publication')
Transformer_serie_publication_histfcast_rescaled[-30:].plot(label='Publication Training Dataset Prediction')
Transformer_serie_publication_pred_rescaled[-30:].plot(label='Publication Validation Dataset Prediction') 

smoothed_occurrence_death_serie[-37:].plot(label='Current Deaths')
Transformer_serie_occurrence_histfcast_rescaled[-30:].plot(label='First Hits Prediction in Training Dataset')
Transformer_serie_occurrence_pred_rescaled[-30:].plot(label='First Hits Prediction in Validation Dataset')

plt.title(label= title_37days + ' - TRANSFORMER Model', fontsize=16)

plt.savefig(PATH_FIGS + title_37days +' - TRANSFORMER_Model.png', dpi = 300)

In [ ]:
#Visualização do alvo de publicação
plt.figure(figsize = (14, 10))

smoothed_publication_death_serie.plot(label='Current Publication')
NHITS_serie_publication_histfcast_rescaled.plot(label='Publication Training Dataset Prediction')
NHITS_serie_publication_pred_rescaled.plot(label='Publication Validation Dataset Prediction') 

smoothed_occurrence_death_serie.plot(label='Current Deaths')
NHITS_serie_occurrence_histfcast_rescaled.plot(label='First Hits Prediction in Training Dataset')
NHITS_serie_occurrence_pred_rescaled.plot(label='First Hits Prediction in Validation Dataset')

plt.title(label= title_7days + ' - NHITS Model', fontsize=16)

plt.savefig(PATH_FIGS + title_7days + ' - NHITS_Model.png', dpi = 300)

In [ ]:

#Visualiação com com os ultimos 37 dias. - Transformer
plt.figure(figsize = (14, 10))

smoothed_publication_death_serie[-37:].plot(label='Current Publication')
NHITS_serie_publication_histfcast_rescaled[-30:].plot(label='Publication Training Dataset Prediction')
NHITS_serie_publication_pred_rescaled[-30:].plot(label='Publication Validation Dataset Prediction') 

smoothed_occurrence_death_serie[-37:].plot(label='Current Deaths')
NHITS_serie_occurrence_histfcast_rescaled[-30:].plot(label='First Hits Prediction in Training Dataset')
NHITS_serie_occurrence_pred_rescaled[-30:].plot(label='First Hits Prediction in Validation Dataset')

plt.title(label= title_37days + ' - NHITS Model', fontsize=16)

plt.savefig(PATH_FIGS + title_37days + ' - NHITS_Model.png', dpi = 300)

#Exportação das predições

In [ ]:
def saving_templates(model, serie_S_HistCast_rescaled, serie_P_HistCast_rescaled, serie_S_Pred_rescaled, serie_P_Pred_rescaled):
  path = os.path.join(PATH_CSV, model.model_name)
  isExist = os.path.exists(path)

  if not isExist:
    os.mkdir(path)

  serie_histfcast = serie_S_HistCast_rescaled.stack(serie_P_HistCast_rescaled)
  df_serie_histfcast_toExport = serie_histfcast.pd_dataframe()
  df_serie_histfcast_toExport.reset_index(inplace=True)
  df_serie_histfcast_toExport.rename(columns = {'time':'data', '0':'Obitos_DataOcorrencia_MM_atual_PA', '1':'Obitos_Publicacao_MM_atual_PA'}, inplace=True)
  df_serie_histfcast_toExport.to_csv(path_or_buf = path +'/training_Forecast.csv', index=False)

  serie_pred_toExport = serie_S_Pred_rescaled.stack(serie_P_Pred_rescaled)
  serie_pred_toExport.to_csv(path_or_buf = path + '/validation_Forecast.csv')


In [ ]:
saving_templates(model_NBEATSModel,
    NBEATS_serie_occurrence_histfcast_rescaled, 
    NBEATS_serie_publication_histfcast_rescaled, 
    NBEATS_serie_occurrence_pred_rescaled,
    NBEATS_serie_publication_pred_rescaled)
print('NBEATS EXPORT')


saving_templates(model_TCNModel,
    TCN_serie_occurrence_histfcast_rescaled, 
    TCN_serie_publication_histfcast_rescaled, 
    TCN_serie_occurrence_pred_rescaled,
    TCN_serie_publication_pred_rescaled)
print('TCN EXPORT')


saving_templates(model_TFTModel,
    TFT_serie_occurrence_histfcast_rescaled, 
    TFT_serie_publication_histfcast_rescaled, 
    TFT_serie_occurrence_pred_rescaled,
    TFT_serie_publication_pred_rescaled)
print('TFT EXPORT')


saving_templates(model_TRANSFORMERModel,
    Transformer_serie_occurrence_histfcast_rescaled, 
    Transformer_serie_publication_histfcast_rescaled, 
    Transformer_serie_occurrence_pred_rescaled,
    Transformer_serie_publication_pred_rescaled)
print('TRANSFORMER EXPORT')


saving_templates(model_NHITSModel,
    NHITS_serie_occurrence_histfcast_rescaled,
    NHITS_serie_publication_histfcast_rescaled, 
    NHITS_serie_occurrence_pred_rescaled,
    NHITS_serie_publication_pred_rescaled)
print('NHITS EXPORT')